In [35]:
import requests
import numpy as np
import pandas as pd

# Function to fetch data based on distance, latitude, and longitude
def fetch_safecast_data(latitude, longitude, date, interval):
    # URL of the JSON API with dynamic variables
    end_date = date + pd.Timedelta(days=interval)
    url = f"https://api.safecast.org/en-US/measurements?captured_after={date.year}%2F{date.month}%2F{date.day}+00%3A00%3A00&captured_before={end_date.year}%2F{end_date.month}%2F{end_date.day}+00%3A00%3A00&format=json&latitude={latitude}&longitude={longitude}"

    response = requests.get(url)


    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse JSON response
        data = response.json()

        # Now, 'data' contains the Python representation of the JSON data
        return data
    else:
        print("Failed to fetch data:", response.status_code)
        return None
    
def fetch_all(latitude_range, longitude_range, date_range, interval):
    columns = ['id', 'user_id', 'value', 'unit', 'location_name', 'device_id', 'original_id', 'measurement_import_id', 'captured_at', 'height', 'devicetype_id', 'sensor_id', 'station_id', 'channel_id', 'latitude', 'longitude']
    result = pd.DataFrame(columns=columns)

    latitude = np.linspace(latitude_range[0], latitude_range[1], interval)
    longitude = np.linspace(longitude_range[0], longitude_range[1], interval)
    dates = pd.date_range(start=date_range[0], end=date_range[0], periods=interval)
    
    for date in dates:
        for lat in latitude:
            for long in longitude:
                batch = pd.DataFrame(fetch_safecast_data(lat, long, date, 1))
                result = pd.concat([result, batch], ignore_index=True)
    return result


Index(['id', 'user_id', 'value', 'unit', 'location_name', 'device_id',
       'original_id', 'measurement_import_id', 'captured_at', 'height',
       'devicetype_id', 'sensor_id', 'station_id', 'channel_id', 'latitude',
       'longitude'],
      dtype='object')


In [39]:
result = fetch_all([34, 35], [134, 135], [pd.Timestamp('2024-03-20'), pd.Timestamp('2024-03-20')], 2)

In [48]:
print(result)

            id user_id  value    unit location_name device_id original_id  \
0    238664070       9   51.0     cpm          None     65008   238664070   
1    238664071       9   43.0     cpm          None     65000   238664071   
2    238664072       9   48.0     cpm          None     65008   238664072   
3    238664076       9   49.0     cpm          None     65000   238664076   
4    238664073       9   41.0     cpm          None    100361   238664073   
..         ...     ...    ...     ...           ...       ...         ...   
195  238664093     759   20.9  status          None    100029   238664093   
196  238664091     759   25.0     cpm          None    100022   238664091   
197  238664094       9   39.0     cpm          None     65008   238664094   
198  238664096     573   16.0     cpm          None       106   238664096   
199  238664095     573   32.0     cpm          None       107   238664095   

    measurement_import_id               captured_at  height  \
0           

In [49]:
print(result[['id', 'value', 'unit', 'height', 'longitude', 'latitude']])

            id  value    unit  height   longitude   latitude
0    238664070   51.0     cpm     NaN  136.163068  34.482693
1    238664071   43.0     cpm     NaN  136.163084  34.482620
2    238664072   48.0     cpm     NaN  136.163063  34.482700
3    238664076   49.0     cpm     NaN  136.163083  34.482623
4    238664073   41.0     cpm   219.0  139.926040  37.494708
..         ...    ...     ...     ...         ...        ...
195  238664093   20.9  status    92.0  139.727850  35.659491
196  238664091   25.0     cpm    92.0  139.727850  35.659491
197  238664094   39.0     cpm     NaN  136.163044  34.482736
198  238664096   16.0     cpm     3.0 -122.128094  37.442836
199  238664095   32.0     cpm     3.0 -122.128094  37.442836

[200 rows x 6 columns]
